In [1]:
# !pip install dendropy

In [46]:
from methods import read_tree, get_tree2_distances, noisify_distances, baseline_method, get_matches, get_scores

# STEP 1 read a tree in Newick format
path = 'trees/test_tree.txt'
# path = 'trees/sym_tree_T1.txt'
path = 'trees/big_tree.txt'
distances1 = read_tree(path)

# STEP 2 change some of the leaves names and add noise to the distances (multiply each distances by a random number between 1 and 1.1)
# changes = {'B': 'X', 'D': 'R', 'F': 'W', 'H': 'Z','E': 'Y'}

n_common_leaves = 2
n_diff_leaves = len(distances1) - n_common_leaves
diff_leaves = distances1.index.to_series().sample(n_diff_leaves, replace=False).to_list()
changes = {c:str(i) for i,c in enumerate(diff_leaves)}

distances2 = get_tree2_distances(distances1, changes)
distances2 = noisify_distances(distances2, 0)

# STEP 3 rank potential matches according to a method.

# Baseline method pairs leaves of T1 with leaves of T2 based on how similar their distance to all common leaves are.
matches = baseline_method(distances1, distances2, epsilon = 0)
# matches = rank_difference_method(distances1, distances2)

# STEP 4 Find the best match in T2 for leaves of T1 and inversely.
t1_matches = get_matches(matches, level=0, changes=changes)
t2_matches = get_matches(matches, level=1, changes=changes)

# STEP 5 compute a score: %age of correct pairs (between 0 and 1)
scores = get_scores(t1_matches, t2_matches)
scores

ratio: 1.0


{'t1_score': 1.0, 't2_score': 1.0, 'mean_score': 1.0}

In [38]:
matches

B  X    0.010519
D  R    0.028948
F  Y    0.032703
   W    0.129665
H  Z    0.042419
dtype: float64

In [ ]:
f(distances1, distances2) -> 

In [37]:
t2_matches

,t1_name,t2_name,distance,t1_match_in_t2
0,B,X,0.010519,X
1,D,R,0.028948,R
2,F,Y,0.032703,W
3,H,Z,0.042419,Z
4,F,W,0.129665,W


In [ ]:
# TODO

# Imaginer et coder une autre methode d'appareillement:
# - Calculer le nombre de permutation dans la liste des n plus proches voisins pour toutes les paires de feuilles
# - Utiliser la correlation de Pearson sur la matrice de distance aux feuilles communes
# - ?

# Trouver des arbres T1 et T2 realistes pour eviter d'utiliser noisify_distances qui n'est pas representative de la realite